In [81]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import socket
import json
import random
import time

In [82]:
# pytorch setup
device = T.device("cuda" if T.cuda.is_available() else "cpu")

# model definition
class DQN(nn.Module):
    
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(n_observations, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, n_actions)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [83]:
TCP_IP = "127.0.0.1"
TCP_PORT = 9876

# start a server
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) # allow reuse of address
sock.bind((TCP_IP, TCP_PORT))

# listen for incoming connections
sock.listen(1)

# keep trying until a connection is established
print("Waiting for connection...")

# setup with godot
conn, addr = sock.accept()
print("Connection established with: ", addr)

# receive the initial message
data = conn.recv(1024)
data = json.loads(data.decode("utf-8"))

if data["type"] == "init":
    print("Initialization message received.")
    numOfObservations = data["numOfObservations"]
    numOfActions = data["numOfActions"]
    print("Number of observations: ", numOfObservations)
    print("Number of actions: ", numOfActions)

Waiting for connection...
Connection established with:  ('127.0.0.1', 36354)
Initialization message received.
Number of observations:  25
Number of actions:  5


In [84]:
# send test observation and reward requests
data = {"type": random.choice(["observe_request"])}
conn.send(json.dumps(data).encode("utf-8"))

observationJSON = conn.recv(4096)
observation = json.loads(observationJSON.decode("utf-8"))
print("Observation received: ", observation["observation"])

data = {"type": random.choice(["reward_request"])}
conn.send(json.dumps(data).encode("utf-8"))

rewardJSON = conn.recv(4096)
reward = json.loads(rewardJSON.decode("utf-8"))
print("Reward received: ", reward["reward"])


JSONDecodeError: Extra data: line 1 column 555 (char 554)

In [ ]:
# send test actions
while True:
    data = {"type": "action", "action": random.choice(["left", "right", "jump"])}
    conn.send(json.dumps(data).encode("utf-8"))
    
    ack = conn.recv(1024)

KeyboardInterrupt: 

In [ ]:
# instantiate the model
model = DQN(numOfObservations, numOfActions).to(device)

# hyperparameters
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
loss = nn.MSELoss()

numIterations = 1000